In [1]:
import os
import pandas as pd
from datetime import datetime

In [2]:
def to_date(df):
    df['data'] = df['ano'].astype(str) + '-' + df['mes'].apply(lambda x: '0' + str(x) if x < 10 else str(x))
    df['data'] = pd.to_datetime(df['data'], format='%Y-%m')

## Carregando e tratando dados da SAGI

In [3]:
files = [f for f in os.listdir("Data/Fam/") if os.path.isfile(os.path.join("Data/Fam/", f))]
total = len(files)
dfs=[]
for file in files:
    df = pd.read_csv(f"Data/Fam/{file}", header=0, index_col='Código')
    df['ano'] = df['Referência'].apply(lambda x: int(x.split('/')[1]))
    df['mes'] = df['Referência'].apply(lambda x: int(x.split('/')[0]))
    df.drop(columns=['Referência'], inplace=True)
    df.set_index(['UF','Unidade Territorial','ano','mes'], inplace=True)
    dfs.append(df)
    print('Arquivo lido. Restam', total-len(dfs), 'arquivos...')
df = pd.concat(dfs, axis=1)


Arquivo lido. Restam 5 arquivos...
Arquivo lido. Restam 4 arquivos...
Arquivo lido. Restam 3 arquivos...
Arquivo lido. Restam 2 arquivos...
Arquivo lido. Restam 1 arquivos...
Arquivo lido. Restam 0 arquivos...


In [5]:
df.rename(columns={
    'Famílias não beneficiárias no Programa Bolsa Família/Programa Auxílio Brasil em situação de extrema pobreza':'fam_nao_benef_ext_pobreza',
    'Famílias beneficiárias no Programa Bolsa Família/Programa Auxílio Brasil em situação de pobreza':'fam_benef_pobreza',
    'Famílias beneficiárias no Programa Bolsa Família/Programa Auxílio Brasil em situação de extrema pobreza':'fam_benef_ext_pobreza',
    'Famílias não beneficiárias no Programa Bolsa Família/Programa Auxílio Brasil em situação de pobreza':'fam_nao_benef_pobreza',
    'Famílias inscritas no Cadastro Único em situação de extrema pobreza':'fam_inscritas_ext_pobreza',
    'Famílias inscritas no Cadastro Único em situação de pobreza':'fam_insc_pobreza',
    },
    inplace = True)

In [6]:
df['demanda_reprimida_total'] = df['fam_nao_benef_pobreza'] + df['fam_nao_benef_ext_pobreza']

In [7]:
uf = df.groupby(['ano','mes','UF']).sum()

In [8]:
br = df.groupby(['ano','mes']).sum()

In [9]:
df.reset_index(inplace=True)
uf.reset_index(inplace=True)
br.reset_index(inplace=True)

In [10]:
uf.columns

Index(['ano', 'mes', 'UF', 'fam_nao_benef_ext_pobreza', 'fam_benef_pobreza',
       'fam_benef_ext_pobreza', 'fam_nao_benef_pobreza',
       'fam_inscritas_ext_pobreza', 'fam_insc_pobreza',
       'demanda_reprimida_total'],
      dtype='object')

In [11]:
to_date(br)
to_date(uf)
to_date(df)

## Carregando e tratando dados do CECAD

In [12]:
siglas = pd.read_csv('Utils/estados_siglas.csv', usecols=['Sigla', 'Estado'])
uf_pes_cecad = pd.read_csv('Data/Pessoas/CECAD/UF_CECAD.csv')
uf_fam_cecad = pd.read_csv('Data/Fam/CECAD/UF_CECAD.csv')
br_fam_cecad = pd.read_csv('Data/Fam/CECAD/BR_CECAD.csv')
br_pes_cecad = pd.read_csv('Data/Pessoas/CECAD/BR_CECAD.csv')

In [13]:
def rename_cols(df, unidade, UF):
    if UF:
        df.columns = ['semnome','UF', f'{unidade}_nao_benef_ext_pobreza',  f'{unidade}_nao_benef_pobreza', f'{unidade}_benef_ext_pobreza', f'{unidade}_benef_pobreza','sr1','sr2',f'{unidade}_inscritas_ext_pobreza', f'{unidade}_insc_pobreza', 'ano', 'mes']
        df.drop(columns=['semnome','sr1','sr2'], inplace=True)
    else:
        df.columns = ['semnome',f'{unidade}_nao_benef_ext_pobreza',  f'{unidade}_nao_benef_pobreza', f'{unidade}_benef_ext_pobreza', f'{unidade}_benef_pobreza','sr1','sr2',f'{unidade}_inscritas_ext_pobreza', f'{unidade}_insc_pobreza', 'ano', 'mes']
        df.drop(columns=['semnome','sr1','sr2'], inplace=True)
    return df

In [14]:
uf_pes_cecad = rename_cols(uf_pes_cecad, 'pessoas', UF=True)
uf_fam_cecad = rename_cols(uf_fam_cecad, 'fam', UF=True)
br_fam_cecad = rename_cols(br_fam_cecad, 'fam', UF=False)
br_pes_cecad = rename_cols(br_pes_cecad, 'pessoas', UF=False)

In [15]:
uf_cecad = pd.merge(uf_pes_cecad, uf_fam_cecad, on=['UF', 'ano', 'mes'])

In [16]:
uf_cecad['UF'] = uf_cecad['UF'].apply(lambda x: x.split('-')[0].strip())

In [17]:
br_cecad = pd.merge(br_pes_cecad, br_fam_cecad, on=['ano', 'mes'])

In [17]:
# uf_cecad = cast_to_int(uf_cecad)
# br_cecad = cast_to_int(br_cecad)

In [18]:
uf_cecad['demanda_reprimida_total'] = uf_cecad['fam_nao_benef_pobreza'] + uf_cecad['fam_nao_benef_ext_pobreza']
br_cecad['demanda_reprimida_total'] = br_cecad['fam_nao_benef_pobreza'] + br_cecad['fam_nao_benef_ext_pobreza']
uf_cecad['demanda_reprimida_pessoas'] = uf_cecad['pessoas_nao_benef_pobreza'] + uf_cecad['pessoas_nao_benef_ext_pobreza']
br_cecad['demanda_reprimida_pessoas'] = br_cecad['pessoas_nao_benef_pobreza'] + br_cecad['pessoas_nao_benef_ext_pobreza']

### Adicionando novas linhas referentes à coleta do CECAD ao dataframe da SAGI

In [19]:
uf = pd.concat([uf,uf_cecad]).merge(siglas, left_on='UF', right_on='Sigla', how='left')
uf.drop(columns=['Sigla'], inplace=True)

In [20]:
br = pd.concat([br,br_cecad])

In [21]:
dt = datetime.date(datetime.now())
br['ts'] = dt

In [22]:
to_date(br)
to_date(uf)

In [23]:
br.tail()

,ano,mes,fam_nao_benef_ext_pobreza,fam_benef_pobreza,fam_benef_ext_pobreza,fam_nao_benef_pobreza,fam_inscritas_ext_pobreza,fam_insc_pobreza,demanda_reprimida_total,data,pessoas_nao_benef_ext_pobreza,pessoas_nao_benef_pobreza,pessoas_benef_ext_pobreza,pessoas_benef_pobreza,pessoas_inscritas_ext_pobreza,pessoas_insc_pobreza,demanda_reprimida_pessoas,ts
113,2022,1,1188947.0,1805132.0,15787132.0,1081238.0,16975529.0,2886349.0,2270185.0,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-06
114,2022,2,1735253.0,1808270.0,15809665.0,1173835.0,17544358.0,2982086.0,2909088.0,2022-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-06
115,2022,3,1995898.0,1824711.0,15824769.0,1210277.0,17820102.0,3034968.0,3206175.0,2022-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-06
116,2022,4,2282276.0,1779297.0,15932367.0,1356972.0,18214051.0,3136245.0,3639248.0,2022-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-06
0,2022,6,2952002.0,1802565.0,15812333.0,1433607.0,18764335.0,3236172.0,4385609.0,2022-06-01,5172792.0,2874586.0,42096191.0,6168408.0,47268983.0,9042994.0,8047378.0,2022-09-06


In [23]:
br.to_parquet('Results/fam_br.parquet')
uf.to_parquet('Results/fam_uf.parquet')
df.to_parquet('Results/fam_mun.parquet')

In [24]:
br.to_parquet('Results/TEST/test_fam_br.parquet')
uf.to_parquet('Results/TEST/test_fam_uf.parquet')
df.to_parquet('Results/TEST/fam_mun.parquet')